**Project Objective:**

In this project, we forecast sales for the next 3 months for 50 different products across 10 different stores using 5 years of store-item sales data. The dataset includes daily sales figures for each store and product. We've applied various techniques to analyze this time series data and predict future demand using machine learning methods.

**Key Highlights:**

- **Model Utilization:** Employed the LightGBM algorithm for sales forecasting. LightGBM is effective in handling large datasets and provides high accuracy.
  
- **Time Series Features:** To enhance model accuracy, we created several key date features:
  - **Exponentially Weighted Mean (EWM):** I computed the exponentially weighted mean on sales data. This method gives more weight to recent data, making the model more sensitive to recent changes and improving prediction accuracy.
  - **Rolling Mean:** I calculated rolling mean to better model trends and seasonality effects.
  - **Lag Features:** I created lag features using past sales data to incorporate historical performance into the model.
  
- **Time Series Analysis:** We processed time series data by considering seasonal patterns, trends, and cyclical effects.

In this project, we achieved high accuracy in sales predictions by applying techniques such as EWM, rolling mean, and lag features to time series data. 

In [0]:
import time
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import lightgbm as lgb
import warnings
from tabulate import tabulate

import plotly.graph_objs as go
import plotly

import sys

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
warnings.filterwarnings('ignore')

In [0]:
'''
This script aims to define custom exception for exception handling purposes. It will be easier to identify errors and issues.
'''

'''
Importing the libraries.
'''

# Debugging and verbose.
import sys


def detailed_error_msg(error, error_details: sys):
    '''
    Generate a detailed error message including file name, line number, and error message.
    
    Args:
        error: The original error or exception.
        error_details (sys): System information about the error.
        
    Returns:
        str: A detailed error message.
    '''
    _, _, exception_traceback = error_details.exc_info()
    file_name = exception_traceback.tb_frame.f_code.co_filename
    line_number = exception_traceback.tb_lineno
    detailed_error_message = f'An error occurred in python file [{file_name}] line number [{line_number}] error message [{str(error)}]'
    
    return detailed_error_message


class CustomException(Exception):
    '''
    Custom exception class with detailed error information.
    '''
    def __init__(self, detailed_error_message: str, error_details: sys) -> None:
        '''
        Initialize a DetailedException instance.

        Args:
            detailed_error_message (str): The detailed error message.
            error_details (sys): System information about the error.
        '''
        super().__init__(detailed_error_message)
        self.detailed_error_message = detailed_error_msg(detailed_error_message, error_details=error_details)


    def __str__(self) -> str:
        '''
        Convert the exception to a string representation.
        
        Returns:
            str: The detailed error message.
        '''
        return self.detailed_error_message
    

In [0]:
def create_time_series_features(data, target, to_sort=None, to_group=None, lags=None, windows=None, weights=None, min_periods=None, win_type=None, date_related=True, lag=False, log_transformation=False, roll=False, ewm=False, roll_mean=False, roll_std=False, roll_min=False, roll_max=False):
    '''
    Create time-series features from the given data.

    Args:
        data (DataFrame): The input data containing time-series information.
        target (str): The name of the target variable.
        to_sort (str, optional): The column name used for sorting the data. Defaults to None.
        to_group (str, optional): The column name used for grouping data. Defaults to None.
        lags (list of int, optional): List of lag values for creating lag features. Defaults to None.
        windows (list of int, optional): List of window sizes for creating rolling window features. Defaults to None.
        weights (list of float, optional): List of weights for creating exponentially weighted mean features. Defaults to None.
        min_periods (int, optional): The minimum number of observations required to have a value. Defaults to None.
        win_type (str, optional): The window type for rolling window calculations. Defaults to None.
        date_related (bool, optional): Flag indicating whether to create date-related features. Defaults to True.
        lag (bool, optional): Flag indicating whether to create lag features. Defaults to False.
        log_transformation (bool, optional): Flag indicating whether to apply log transformation to the target variable. Defaults to False.
        roll (bool, optional): Flag indicating whether to create rolling window features. Defaults to False.
        ewm (bool, optional): Flag indicating whether to create exponentially weighted mean features. Defaults to False.
        roll_mean (bool, optional): Flag indicating whether to create rolling mean features. Defaults to False.
        roll_std (bool, optional): Flag indicating whether to create rolling standard deviation features. Defaults to False.
        roll_min (bool, optional): Flag indicating whether to create rolling minimum features. Defaults to False.
        roll_max (bool, optional): Flag indicating whether to create rolling maximum features. Defaults to False.

    Returns:
        DataFrame: DataFrame containing the original data with additional time-series features.

    Raises:
        CustomException: If an exception occurs during feature creation.
    '''
    try:
        df = data.copy()

        # Create date-related features.
        if date_related:
            df['dayofweek'] = df.index.dayofweek
            df['quarter'] = df.index.quarter
            df['month'] = df.index.month
            df['year'] = df.index.year
            df['dayofyear'] = df.index.dayofyear
            df['dayofmonth'] = df.index.day
            df['weekofyear'] = df.index.isocalendar().week.astype(np.float64)
            df['is_wknd'] = df.index.weekday // 4
            df['is_month_start'] = df.index.is_month_start.astype(int)
            df['is_month_end'] = df.index.is_month_end.astype(int)

        # Apply log_transformation to the target variable.
        if log_transformation:
            df[target] = np.log1p(df[target])
        
        # Create lag features.
        if lag:
            df.sort_values(by=to_sort, axis=0, inplace=True)
            for lag in lags:
                df['sales_lag_' + str(lag)] = df.groupby(to_group)[target].transform(lambda x: x.shift(lag))
        
        # Create rolling window features.
        if roll:
            df.sort_values(by=to_sort, axis=0, inplace=True)

            if roll_mean:
                for window in windows:
                    df['sales_roll_mean_' + str(window)] = df.groupby(to_group)[target].transform(lambda x: x.shift(1).rolling(window=window, min_periods=min_periods, win_type=win_type).mean())
            if roll_std:
                for window in windows:
                    df['sales_roll_std_' + str(window)] = df.groupby(to_group)[target].transform(lambda x: x.shift(1).rolling(window=window, min_periods=min_periods, win_type=win_type).std())
            if roll_min:
                for window in windows:
                    df['sales_roll_min_' + str(window)] = df.groupby(to_group)[target].transform(lambda x: x.shift(1).rolling(window=window, min_periods=min_periods, win_type=win_type).min())
            if roll_max:
                for window in windows:
                    df['sales_roll_max_' + str(window)] = df.groupby(to_group)[target].transform(lambda x: x.shift(1).rolling(window=window, min_periods=min_periods, win_type=win_type).max())

        # Create exponentially weighted mean features.
        if ewm:
            for weight in weights:
                    for lag in lags:
                        df['sales_ewm_w_' + str(weight) + '_lag_' + str(lag)] = df.groupby(to_group)[target].transform(lambda x: x.shift(lag).ewm(alpha=weight).mean())
            
        return df

    except Exception as e:
        raise CustomException(e, sys)

def time_series_split(data, cutoff_date):
    '''
    Splits the time series data into train and test sets on a chronological order based on the cutoff date.

    Args:
    data (pandas.DataFrame): The time series data to be split.
    cutoff_date (str or datetime): The date that separates the training and test sets.

    Raises:
    CustomException: An error occurred during the time series split.

    Returns:
    tuple: A tuple containing two pandas.DataFrame objects, where the first one represents the training set
    with data before the cutoff date, and the second one represents the test set with data on and after the cutoff date.
    '''
    try:
        train = data.loc[data.index < cutoff_date]
        test = data.loc[data.index >= cutoff_date]
        return train, test
    
    except Exception as e:
        raise CustomException(e, sys)
    

def plot_time_series_split(train, test, cutoff_date):
    '''
    Plots the time series data after splitting into train and test sets.

    Args:
    train (pandas.DataFrame): The training data to be plotted.
    test (pandas.DataFrame): The test data to be plotted.
    cutoff_date (str or datetime): The date that separates the training and test sets.

    Raises:
    CustomException: An error occurred during the plotting process.
    '''
    try:
        figure, ax = plt.subplots(figsize=(20, 7))

        train.plot(ax=ax, label='Train', y='sales')
        test.plot(ax=ax, label='Test', y='sales')

        ax.axvline(cutoff_date, color='black', ls='--')

        plt.title('Time series train-test-split', fontsize=25, fontweight='bold', loc='left', pad=25)
        plt.xlabel('Date', loc='left', labelpad=25)
        plt.ylabel('Sales', loc='top', labelpad=25)
        plt.xticks(rotation=0)
        plt.legend(loc='upper left')
        plt.show()
    
    except Exception as e:
        raise CustomException(e, sys)

In [0]:
import statsmodels.api as sm
from sklearn.model_selection import TimeSeriesSplit

#SET GLOBAL FLAG
should_continue_execution = True  # Default to True

# Utils.
#from src.modelling_utils import create_time_series_features, time_series_split, plot_time_series_split


In [0]:
#Widgets used to pass value
dbutils.widgets.text("job-id", "100")
dbutils.widgets.text("postback-url", "")
dbutils.widgets.text("rb_detail", "")
dbutils.widgets.text("rb_summary", "")
dbutils.widgets.text("rb_detail_org", "")

#dbutils.widgets.text("bt_insights", "N")

jobId = dbutils.widgets.get("job-id")
rb_detail = dbutils.widgets.get("rb_detail")
rb_features = dbutils.widgets.get("rb_detail")
rb_detail_org = dbutils.widgets.get("rb_detail_org")
if rb_detail_org == '3':
    rb_detail = '3'
rb_summary = dbutils.widgets.get("rb_summary")
webserverURL = dbutils.widgets.get("postback-url")
bt_insights = rb_detail
#print(f"parameter_value for key rb_summary is: {rb_summary} and parameter_value for key rb_detail is: {rb_detail}")
#print(f"parameter_value for key rb_features is: {rb_features}")
#Initialize and Start Execution
from fire_notebook.output.workflowcontext import RestWorkflowContext
restworkflowcontext = RestWorkflowContext(webserverURL, jobId)
message="20"
restworkflowcontext.outputProgress(9, title="Progress", progress=message)

Sending Message: {"progress": "20", "id": 9, "name": "Progress", "title": "Progress", "type": "progress", "resultType": 3, "visibility": "EXPANDED", "time": "Oct 10, 2024 05:34:54 PM"}
Not sending message to fire_notebook server as the post back URL is not http

### EDA Functions

In [0]:
# Define the check_df function
def check_df(dataframe: pd.DataFrame, head: int = 5) -> None:
    # Print DataFrame shape
    htmlstr1 = f"""
    <div style="border: 1px solid #ddd; padding: 10px; margin-bottom: 10px; background-color: #f9f9f9;">
        <h3>DataFrame Shape</h3>
        <p>Rows: {dataframe.shape[0]}, Columns: {dataframe.shape[1]}</p>
    </div>
    """
    restworkflowcontext.outHTML(9, title="DataFrame Shape", text=htmlstr1)

    # Print DataFrame unique values in HTML format
    unique_values = dataframe.nunique().reset_index()
    unique_values.columns = ['Column', 'Unique Values']
    unique_values_html = unique_values.to_html(index=False, border=0, classes='table table-striped')


    # Extract only the <tbody> section from the generated HTML
    tbody_start = unique_values_html.find('<tbody>')
    tbody_end = unique_values_html.find('</tbody>') + len('</tbody>')
    unique_values_tbody = unique_values_html[tbody_start:tbody_end]

    unique_values_html_full = f"""
    <div style="border: 1px solid #ddd; padding: 10px; margin-bottom: 10px; background-color: #f9f9f9;">
        <table class="table table-striped" id="unique_values_table" style="width: 100%; text-align: left;">
            <thead style="text-align: left;">
                <tr>
                    <th>Column</th>
                    <th>Unique Values</th>
                </tr>
            </thead>
            {unique_values_tbody}
        </table>
    </div>
    """
    restworkflowcontext.outHTML(10, title="Unique Values Per Column", text=unique_values_html_full)

    # Print DataFrame data types
    data_types = dataframe.dtypes.reset_index()
    data_types.columns = ['Column', 'Data Type']
    data_types_html = data_types.to_html(index=False, border=0, classes='table table-striped')

    # Extract only the <tbody> section from the generated HTML
    tbody_start = data_types_html.find('<tbody>')
    tbody_end = data_types_html.find('</tbody>') + len('</tbody>')
    data_types_tbody = data_types_html[tbody_start:tbody_end]

    data_types_html_full = f"""
    <div style="border: 1px solid #ddd; padding: 10px; margin-bottom: 10px; background-color: #f9f9f9;">
        <table class="table table-striped" id="data_types_table" style="width: 100%; text-align: left;">
            <thead style="text-align: left;">
                <tr>
                    <th>Column</th>
                    <th>Data Type</th>
                </tr>
            </thead>
            {data_types_tbody}
        </table>
    </div>
    """
    restworkflowcontext.outHTML(11, title="Data Types", text=data_types_html_full)

    # Print head of the DataFrame
    head_html = dataframe.head().to_html(index=False, border=0, classes='table table-striped')

    # Extract only the <tbody> section from the generated HTML
    tbody_start = head_html.find('<tbody>')
    tbody_end = head_html.find('</tbody>') + len('</tbody>')
    head_tbody = head_html[tbody_start:tbody_end]

    head_html_full = f"""
    <div style="border: 1px solid #ddd; padding: 10px; margin-bottom: 10px; background-color: #f9f9f9;">
        <table class="table table-striped" id="head_table" style="width: 100%; text-align: left;">
            <thead style="text-align: left;">
                <tr>
                    {"".join(f"<th>{col}</th>" for col in dataframe.columns)}
                </tr>
            </thead>
            {head_tbody}
        </table>
    </div>
    """
    restworkflowcontext.outHTML(12, title="Top Rows (Head)", text=head_html_full)

    # Print tail of the DataFrame
    tail_html = dataframe.tail().to_html(index=False, border=0, classes='table table-striped')

    # Extract only the <tbody> section from the generated HTML
    tbody_start = tail_html.find('<tbody>')
    tbody_end = tail_html.find('</tbody>') + len('</tbody>')
    tail_tbody = tail_html[tbody_start:tbody_end]

    tail_html_full = f"""
    <div style="border: 1px solid #ddd; padding: 10px; margin-bottom: 10px; background-color: #f9f9f9;">
        <table class="table table-striped" id="tail_table" style="width: 100%; text-align: left;">
            <thead style="text-align: left;">
                <tr>
                    {"".join(f"<th>{col}</th>" for col in dataframe.columns)}
                </tr>
            </thead>
            {tail_tbody}
        </table>
    </div>
    """
    restworkflowcontext.outHTML(13, title="Bottom Rows (Tail)", text=tail_html_full)    

    # Print missing values
    missing_values = dataframe.isnull().sum().reset_index()
    missing_values.columns = ['Column', 'Missing Values']
    missing_values_html = missing_values.to_html(index=False, border=0, classes='table table-striped')

# Extract only the <tbody> section from the generated HTML
    tbody_start = missing_values_html.find('<tbody>')
    tbody_end = missing_values_html.find('</tbody>') + len('</tbody>')
    missing_values_tbody = missing_values_html[tbody_start:tbody_end]

    missing_values_html_full = f"""
    <div style="border: 1px solid #ddd; padding: 10px; margin-bottom: 10px; background-color: #f9f9f9;">
        <table class="table table-striped" id="missing_values_table" style="width: 100%; text-align: left;">
            <thead style="text-align: left;">
                <tr>
                    <th>Column</th>
                    <th>Missing Values</th>
                </tr>
            </thead>
            {missing_values_tbody}
        </table>
    </div>
    """
    restworkflowcontext.outHTML(14, title="Missing Values Per Column", text=missing_values_html_full)

    # Print date range if 'date' column exists
    if 'date' in dataframe.columns and pd.api.types.is_datetime64_any_dtype(dataframe['date']):
        date_range_html = f"""
        <div style="border: 1px solid #ddd; padding: 10px; margin-bottom: 10px; background-color: #f9f9f9;">
            <h3>Date Range</h3>
            <p>Start Date: {dataframe['date'].min()}</p>
            <p>End Date: {dataframe['date'].max()}</p>
        </div>
        """
        restworkflowcontext.outHTML(15, title="Date Range", text=date_range_html)

    # Print quantile statistics
    quantiles = dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T.reset_index()
    quantiles_html = quantiles.to_html(index=False, border=0, classes='table table-striped')

    # Extract only the <tbody> section from the generated HTML
    tbody_start = quantiles_html.find('<tbody>')
    tbody_end = quantiles_html.find('</tbody>') + len('</tbody>')
    quantiles_tbody = quantiles_html[tbody_start:tbody_end]

    quantiles_html_full = f"""
    <div style="border: 1px solid #ddd; padding: 10px; margin-bottom: 10px; background-color: #f9f9f9;">
        <table class="table table-striped" id="quantiles_table" style="width: 100%; text-align: left;">
            <thead style="text-align: left;">
                <tr>
                    {"".join(f"<th>{col}</th>" for col in quantiles.columns)}
                </tr>
            </thead>
            {quantiles_tbody}
        </table>
    </div>
    """
    restworkflowcontext.outHTML(16, title="Quantiles", text=quantiles_html_full)


# target summary with cat cols
# Function to see the average sales for each store and the average sales for each item.
#Example Business Questions that can be answered:
    #Which store has the highest average sales value?
    #Which item has the lowest average sales value?

def target_summary_with_cat(dataframe, target, categorical_cols):
    for col in categorical_cols:
        summary_df = dataframe.groupby(col)[target].mean().reset_index()
        summary_df.columns = [col, "TARGET_MEAN"]

        # Convert DataFrame to HTML
        summary_html = summary_df.to_html(index=False, border=0, classes='table table-striped')

        # Extract <tbody> section (optional, for styling)
        tbody_start = summary_html.find('<tbody>')
        tbody_end = summary_html.find('</tbody>') + len('</tbody>')
        summary_tbody = summary_html[tbody_start:tbody_end]

        # Wrap HTML in a div for styling (optional)
        summary_html_full = f"""
        <div style="border: 1px solid #ddd; padding: 10px; margin-bottom: 10px; background-color: #f9f9f9;">
            <table class="table table-striped" id="summary_table" style="width: 100%; text-align: left;">
                <thead>
                    <tr>
                        <th>{col}</th>
                        <th>TARGET_MEAN</th>
                    </tr>
                </thead>
                {summary_tbody}
            </table>
        </div>
        """
        restworkflowcontext.outHTML(16, title="Detailed Report", text=summary_html_full)

        print(summary_df, end="\n\n\n")

        # Create histogram
        plt.figure(figsize=(15, 6))
        plt.bar(summary_df[col].astype(str), summary_df["TARGET_MEAN"], color='skyblue')
        plt.xlabel(col)
        plt.ylabel(target)
        plt.title(f"{target} mean for {col}")
        plt.xticks(rotation=45)
        plt.show()
        #fig = plotly.offline.plot([go.Bar(x=summary_df[col], y=summary_df["TARGET_MEAN"])], output_type='div', include_plotlyjs=False)
        #example_plotly = f'{fig}'
        #restworkflowcontext.outPlotly(9, title="MEAN OF", text=example_plotly )
        #fig = go.Figure(data=[go.Bar(x=summary_df[col], y=summary_df["TARGET_MEAN"])])
        fig = go.Figure(data=[go.Bar(x=summary_df[col], y=summary_df["TARGET_MEAN"], marker_color='skyblue')])
        fig.update_layout(
            #title=f"{target} mean for {col}",
            xaxis_title=col,
            yaxis_title=target
        )

        # Display the chart (replace with your output method)
        fig_html = plotly.offline.plot(fig, output_type='div', include_plotlyjs=False)
        restworkflowcontext.outPlotly(9, title= f"{target.upper()} MEAN FOR {col.upper()}", text=fig_html )


__Loading the data__

In [0]:
train = pd.read_csv('/dbfs/FileStore/Demand-Forecasting/Input/demand-train.csv', parse_dates=['date'])
test = pd.read_csv('/dbfs/FileStore/Demand-Forecasting/Input/demand-test.csv', parse_dates=['date'])

#sample_sub = pd.read_csv('/dbfs/FileStore/Demand-Forecasting/Input/demand_forecasting.csv')

df = pd.concat([train, test], sort=False)


%md
### Time series decomposition
-  Time series data is a sequence of data points indexed in time order, typically at uniform intervals, used to track changes over time and analyze trends, patterns, and seasonal variations.
- A time series is stationary when its statistical characteristics, such as mean, variance and covariance don't change over time.
- Time series decomposition: Analyze time series data by breaking it down into components to understand the underlying patterns, trends and irregularities within the data.
- Trend, seasonal, cyclical and residual components.
- Trend Component: Underlying long-term progression or direction of the time series. Shows wheter the data is increasing, decreasing or remaining relatively constant over time.
- Seasonal Component: Regular, periodic fluctuations or patterns that occurs at specific intervals within the time series (annually, quarterly, monthly, and so on).
- Cyclical Component: Fluctuations in the time series that are not of a fixed period. Typically associated with business/economic cycles.
- Residual Component: Random fluctuations or irregularities that cannot be attributed to the trend, seasonal or cyclical patterns. Unexplained variability in the time series.
- In order to perform time series decomposition, I will use the statsmodels package, applying seasonal decomposition using moving averages.
- Moving Average: A moving average is a method that smooths data by creating averages from subsets of consecutive data points, providing a clearer picture of trends and patterns while reducing noise and short-term fluctuations.
- Approaches used to decompose a time series: Additive Model, Multiplicative Model.
- Additive Model: Time Series = Trend + Seasonal + Cyclical + Residual. The relation between the components is linear, constant variance over time.
- Multiplicative Model: Time Series = Trend * Seasonal * Cyclical * Residual. The relation between the components is non-linear, variance is not constant over time, changing with the level of the series (average value around which the data fluctuates).

__Building Modular Functions__

In [0]:
def run_analysis(dataframe, options):
    global should_continue_execution 
    # Retrieve widget values
    rb_summary = dbutils.widgets.get("rb_summary")
    rb_detail = dbutils.widgets.get("rb_detail")

    # Ensure rb_detail is a string for consistent comparison
    if isinstance(rb_detail, int):
        rb_detail = str(rb_detail)

    # Debug print statement
    print(f"Retrieved rb_summary: {rb_summary} and rb_detail: {rb_detail}")

    # Check if both rb_summary and rb_detail are set to 'Y' and '1' respectively
    if rb_summary == 'Y' and rb_detail == '1':
        check_df(dataframe)
        target_summary_with_cat(
            dataframe,
            options['target'],
            options['categorical_col']
        )
        should_continue_execution = False
        return  # Stop further execution

    # Check if only rb_summary is set to 'Y'
    if rb_summary == 'Y':
        check_df(dataframe)
        should_continue_execution = False
        return  # Stop further execution

    # Check if only rb_detail is set to '1'
    if rb_detail == '1':
        target_summary_with_cat(
            dataframe,
            options['target'],
            options['categorical_col']
        )
        should_continue_execution = False
        return  # Stop further execution

    if rb_detail == '2' and rb_summary == 'N':
        should_continue_execution = True  # Ensure execution continues to the next cell
        print("Skipping this cell based on rb_detail and rb_summary values.")
        return  # Exit the function, skipping the rest of the cell



# Example usage of the function
dataframe = df
options = {
    'target': 'sales',           # Replace 'sales' with the actual target column name in your DataFrame
    'categorical_col': ['store', 'item']  # Replace 'store' and 'item' with the actual categorical column names in your DataFrame
}

# Call the run_analysis function
run_analysis(dataframe, options)

if not should_continue_execution:
    message="100"
    restworkflowcontext.outputProgress(9, title="Progress", progress=message)

    message = "Job Execution Completed."
    restworkflowcontext.outSuccess(9, title="Success", text=message)    
    dbutils.notebook.exit("Execution stopped by run_analysis function.")
else:
    print(f"First  values: rb_detail={rb_detail}, rb_summary={rb_summary}") 



In [0]:
# Function to provide a more comprehensive understanding of sales performance.
#Total Sales: See which store-item combinations generate the highest overall revenue.
#Average Sales: Understand typical sales performance for each combination.
#Median Sales: Get a sense of the "typical" sale amount, less influenced by outliers.

#Example Business Questions that can be answered:
  #Which store-item pair has the highest total sales value?
  #Which store-item pair has the highest average sales value?
  #Which store-item pair has the most consistent sales values (i.e., lowest standard deviation)?



def generate_detailed_sales_summary(df):
    summary_df = df.groupby(["store", "item"]).agg({"sales": ["sum", "mean", "median", "std"]})

    # Find the store-item pair with the highest total sales value
    highest_total_sales = summary_df['sales']['sum'].nlargest(1)

    # Find the store-item pair with the highest average sales value
    highest_average_sales = summary_df['sales']['mean'].nlargest(1)

    # Find the store-item pair with the most consistent sales values (i.e., lowest standard deviation)
    most_consistent_sales = summary_df['sales']['std'].nsmallest(1)


    print(f"Aggregates: highest_total_sales={highest_total_sales}, highest_average_sales={highest_average_sales}") 

    # Create HTML string with styled output
    html_output = f"""
    <div style="font-family: Arial, sans-serif; background-color: #f5f5f5; padding: 20px; border-radius: 5px;">
        <h2 style="color: #333; font-size: 24px; margin-bottom: 20px;">Sales Analysis For Store-Item pair</h2>
        <table style="width: 100%; border-collapse: collapse;">
            <thead>
                <tr style="background-color: #0c9a86; color: white;">
                    <th style="padding: 10px; text-align: left;">Metric</th>
                    <th style="padding: 10px; text-align: left;">Store</th>
                    <th style="padding: 10px; text-align: left;">Item</th>
                    <th style="padding: 10px; text-align: left;">Value</th>
                </tr>
            </thead>
            <tbody>
                <tr style="background-color: #f2f2f2;">
                    <td style="padding: 8px; text-align: left;">Highest Total Sales</td>
                    {''.join([f'<td style="padding: 8px; text-align: left;">{int(v) if i in [1, 2] else v}</td>' for i, v in enumerate(highest_total_sales.reset_index().iloc[0].values)])}
                </tr>
                <tr>
                    <td style="padding: 8px; text-align: left;">Highest Average Sales</td>
                    {''.join([f'<td style="padding: 8px; text-align: left;">{int(v) if i in [1, 2] else v}</td>' for i, v in enumerate(highest_average_sales.reset_index().iloc[0].values)])}
                </tr>
                <tr style="background-color: #f2f2f2;">
                    <td style="padding: 8px; text-align: left;">Most Consistent Sales</td>
                    {''.join([f'<td style="padding: 8px; text-align: left;">{int(v) if i in [1, 2] else v}</td>' for i, v in enumerate(most_consistent_sales.reset_index().iloc[0].values)])}
                </tr>
            </tbody>
        </table>
    </div>
    """
    #print(html_output)
    restworkflowcontext.outHTML(16, title="Inital Insights", text=html_output)

    should_continue_execution = False
    return  # Stop further execution

if rb_detail  == '2':
    # Call the function
    generate_detailed_sales_summary(df)
    should_continue_execution = False
    message="100"
    restworkflowcontext.outputProgress(9, title="Progress", progress=message)
    message = "Job Execution Completed."
    restworkflowcontext.outSuccess(9, title="Success", text=message)    
    dbutils.notebook.exit("Execution stopped by generate_detailed_sales_summary function.")    
  
elif rb_detail == '3' and rb_summary == 'N':
    should_continue_execution = True  # Ensure execution continues to the next cell
    print("Skipping this cell based on rb_detail and rb_summary values.")
  
else:
    # Exit the notebook if bt_insights is not 'Y'
    message="100"
    restworkflowcontext.outputProgress(9, title="Progress", progress=message)
    message = "Job Execution Completed."
    restworkflowcontext.outSuccess(9, title="Success", text=message)    
    dbutils.notebook.exit("Execution stopped by generate_detailed_sales_summary function.")
    

Skipping this cell based on rb_detail and rb_summary values.

# 2) FEATURE ENGINEERING

## a) Date Features
Since it will be used with tree-based methods, we created new features based on the date variable.

In [0]:
def create_date_features(df):
    df['month'] = df.date.dt.month
    df['day_of_month'] = df.date.dt.day
    df['day_of_year'] = df.date.dt.dayofyear
    df['day_of_week'] = df.date.dt.dayofweek
    df['year'] = df.date.dt.year
    df["is_wknd"] = df.date.dt.weekday // 4
    df['is_month_start'] = df.date.dt.is_month_start.astype(int)
    df['is_month_end'] = df.date.dt.is_month_end.astype(int)
    return df

#df = create_date_features(df)


#Visualizes average sales data at different aggregation levels with optional top N filtering.#

In [0]:
#Average sales for each group.
#df.groupby(["store", "item", "month"])["sales"].mean().head(36)

def plot_sales(df, plot_type="bar", agg_level="month", top_n=None):
    """
    Plots average sales data, optionally showing the top N combinations.
    """
    # Group by the specified level
    if agg_level == "month":
        sales_df = df.groupby("month")["sales"].mean().reset_index()
        xlabel = "Month"
    elif agg_level == "store":
        sales_df = df.groupby("store")["sales"].mean().reset_index()
        xlabel = "Store"
    elif agg_level == "item":
        sales_df = df.groupby("item")["sales"].mean().reset_index()
        xlabel = "Item"
    elif agg_level == "store-item":
        sales_df = df.groupby(["store", "item"])["sales"].mean().reset_index()
        sales_df['store-item'] = sales_df['store'].astype(str) + '-' + sales_df['item'].astype(str)
        xlabel = "Store-Item Pair"
    else:
        raise ValueError("Invalid aggregation level. Choose from: 'month', 'store', 'item', 'store-item'")

     # Filter out the aggregation levels with no sales
    sales_df = sales_df[sales_df["sales"] > 0]
    # Sort the data by sales in descending order and select the top n combinations
    if sales_df.shape[0] < top_n:
        top_n = sales_df.shape[0]
    sales_df = sales_df.sort_values(by="sales", ascending=False).head(top_n)    

    # Select top N combinations if top_n is specified
    #if top_n is not None:
    #    sales_df = sales_df.sort_values(by="sales", ascending=False).head(top_n)

    # Create the plot
    plt.figure(figsize=(12, 6))
    if plot_type == "bar":
        if agg_level == "store-item":
            plt.bar(sales_df['store-item'], sales_df["sales"])
        else:
            plt.bar(sales_df[agg_level], sales_df["sales"])
    elif plot_type == "line":
        if agg_level == "store-item":
            plt.plot(sales_df['store-item'], sales_df["sales"])
        else:
            plt.plot(sales_df[agg_level], sales_df["sales"])
    else:
        raise ValueError("Invalid plot type. Choose from: 'bar', 'line'")

    plt.xlabel(xlabel)
    plt.ylabel("Average Sales")
    plt.title(f"Average Sales by {xlabel.title()} (Top {top_n})" if top_n else f"Average Sales by {xlabel.title()}")
    plt.xticks(rotation=90 if agg_level == "store-item" else 0) 
    plt.tight_layout()
    plt.show()


#plot_sales(df, plot_type="bar", agg_level="month", top_n=5) # Show top 5 store-item combo
#plot_sales(df, plot_type="bar", agg_level="store", top_n=5) # Show top 5 store-item combo
#plot_sales(df, plot_type="bar", agg_level="item", top_n=5) # Show top 5 store-item combo
#plot_sales(df, plot_type="bar", agg_level="store-item", top_n=5) # Show top 5 store-item combo



## b) Random Noise

Random noise should be added to evaluate the overall performance of the model and to prevent overfitting

In [0]:
'''
def random_noise(dataframe):
    return np.random.normal(scale=1.6, size=(len(dataframe),))
'''
def random_noise(dataframe, feature_flags):
    return np.random.normal(scale=1.6, size=(len(dataframe),)) if feature_flags['random_noise'] else np.zeros(len(dataframe))

## c) Lag/Shifted Features

The purpose of lag features is to use past data points to predict future values. They help the model learn patterns and dependencies over time, improving the accuracy of forecasts.

In [0]:
def lag_features(dataframe, lags, feature_flags):
    for lag in lags:
        dataframe[f'sales_lag_{lag}'] = dataframe.groupby(["store", "item"])['sales'].transform(
            lambda x: x.shift(lag)) + random_noise(dataframe, feature_flags)
    return dataframe

'''
def lag_features(dataframe, lags):
    for lag in lags:
        dataframe['sales_lag_' + str(lag)] = dataframe.groupby(["store", "item"])['sales'].transform(
            lambda x: x.shift(lag)) + random_noise(dataframe)
    return dataframe
df = lag_features(df, [91, 98, 105, 112, 119, 126, 182, 364, 546, 728])
'''

Out[13]: '\ndef lag_features(dataframe, lags):\n for lag in lags:\n dataframe[\'sales_lag_\' + str(lag)] = dataframe.groupby(["store", "item"])[\'sales\'].transform(\n lambda x: x.shift(lag)) + random_noise(dataframe)\n return dataframe\ndf = lag_features(df, [91, 98, 105, 112, 119, 126, 182, 364, 546, 728])\n'

## d) Rolling Mean Features
Rolling mean features help to smooth out short-term fluctuations in time series data, making it easier to identify long-term trends and patterns.

In [0]:
def roll_mean_features(dataframe, windows, feature_flags):
    for window in windows:
        dataframe[f'sales_roll_mean_{window}'] = dataframe.groupby(["store", "item"])['sales']. \
                                                          transform(
            lambda x: x.shift(1).rolling(window=window, min_periods=10, win_type="triang").mean()) + random_noise(dataframe, feature_flags)
    return dataframe

'''
def roll_mean_features(dataframe, windows):
    for window in windows:
        dataframe['sales_roll_mean_' + str(window)] = dataframe.groupby(["store", "item"])['sales']. \
                                                          transform(
            lambda x: x.shift(1).rolling(window=window, min_periods=10, win_type="triang").mean()) + random_noise(
            dataframe)
    return dataframe
'''

#df = roll_mean_features(df, [365, 546])

Out[14]: '\ndef roll_mean_features(dataframe, windows):\n for window in windows:\n dataframe[\'sales_roll_mean_\' + str(window)] = dataframe.groupby(["store", "item"])[\'sales\']. transform(\n lambda x: x.shift(1).rolling(window=window, min_periods=10, win_type="triang").mean()) + random_noise(\n dataframe)\n return dataframe\n'

## e) Exponentially Weighted Mean Features
* __EWMA (Exponential Weighted Moving Average):__ Computes the weighted moving average of the data, where more recent data points receive higher weights.
* __Lag Features:__ Use past data to calculate the EWMA of historical values, helping to understand the dynamics of the data over time.
* __Alpha Parameter:__ Determines the weight given to past periods. Higher alpha values place more emphasis on recent data.


In [0]:

def ewm_features(dataframe, alphas, lags):
    for alpha in alphas:
        for lag in lags:
            dataframe[f'sales_ewm_alpha_{alpha}_lag_{lag}'] = \
                dataframe.groupby(["store", "item"])['sales'].transform(lambda x: x.shift(lag).ewm(alpha=alpha).mean())
    return dataframe

'''
def ewm_features(dataframe, alphas, lags):
    for alpha in alphas:
        for lag in lags:
            dataframe['sales_ewm_alpha_' + str(alpha).replace(".", "") + "_lag_" + str(lag)] = \
                dataframe.groupby(["store", "item"])['sales'].transform(lambda x: x.shift(lag).ewm(alpha=alpha).mean())
    return dataframe
'''
alphas = [0.95, 0.9, 0.8, 0.7, 0.5]
lags = [91, 98, 105, 112, 180, 270, 365, 546, 728]

#df = ewm_features(df, alphas, lags)

## f) One-Hot Encoding
We performed one-hot encoding because, as observed in the data analysis, the sales amounts for each store and item vary. To help our model understand this better, we used one-hot encoding with get_dummies."

In [0]:
#df = pd.get_dummies(df, columns=['store', 'item', 'day_of_week', 'month'])

## g) Converting sales to log(1+sales)
We applied a logarithmic transformation using log(1+sales) to make the model run more efficiently.

In [0]:
#df['sales'] = np.log1p(df["sales"].values)

In [0]:
df

Out[18]:

,date,store,item,sales,id
0,2013-01-01,1,1,13.0,NaN
1,2013-01-02,1,1,11.0,NaN
2,2013-01-03,1,1,14.0,NaN
3,2013-01-04,1,1,13.0,NaN
4,2013-01-05,1,1,10.0,NaN
...,...,...,...,...,...
44995,2018-03-27,10,50,NaN,44995.0
44996,2018-03-28,10,50,NaN,44996.0
44997,2018-03-29,10,50,NaN,44997.0
44998,2018-03-30,10,50,NaN,44998.0


Feature Engineering Modular Functions

In [0]:
#print("rb_detail", rb_features)  


flag_mapping = {
    'RF': 'random_noise',
    'LF': 'lag_features',
    'RMF': 'rolling_mean_features',
    'EF': 'ewm_features',
    'OHE': 'one_hot_encoding',
    'LT': 'log_transform'
}

# Split rb_features by '|' and clean the flags
selected_flags = [flag_mapping.get(flag.strip().upper()) for flag in rb_features.split('|') if flag.strip().upper() in flag_mapping]

# Set the feature_flags based on the selected_flags
feature_flags = {flag: True for flag in selected_flags}
for flag in flag_mapping.values():
    if flag not in selected_flags:
        feature_flags[flag] = False

#print("Before" , df.columns)


def feature_engineering(df, feature_flags):
    if 'date' not in df.columns:
        raise ValueError("Date column not found in DataFrame")

    df = create_date_features(df)

    original_columns = df.columns

    if feature_flags['one_hot_encoding']:
        columns_to_encode = ['store', 'item', 'day_of_week', 'month']
        columns_to_encode = [col for col in columns_to_encode if col in df.columns]
        df = pd.get_dummies(df, columns=columns_to_encode, prefix_sep='_')

    if 'store' not in df.columns or 'item' not in df.columns:
        if 'store' not in df.columns:
            store_columns = [col for col in df.columns if col.startswith('store_')]
            if store_columns:
                df['store'] = df[store_columns].idxmax(axis=1).str.split('_').str[1]
        if 'item' not in df.columns:
            item_columns = [col for col in df.columns if col.startswith('item_')]
            if item_columns:
                df['item'] = df[item_columns].idxmax(axis=1).str.split('_').str[1]

    if feature_flags['lag_features']:
        df = lag_features(df, [91, 98, 105, 112, 119, 126, 182, 364, 546, 728], feature_flags)

    if feature_flags['rolling_mean_features']:
        df = roll_mean_features(df, [365, 546], feature_flags)

    if feature_flags['ewm_features']:
        alphas = [0.95, 0.9, 0.8, 0.7, 0.5]
        lags = [91, 98, 105, 112, 180, 270, 365, 546, 728]
        df = ewm_features(df, alphas, lags)

    if feature_flags['log_transform']:
        df['sales'] = np.log1p(df["sales"].values)

    return df

# Assuming `df` is your DataFrame before feature engineering
if rb_detail == '3':
    # Call the function
    #df, html_output = feature_engineering(df, feature_flags)
    df = feature_engineering(df, feature_flags)
    
    #df.to_csv("/dbfs/FileStore/Demand-Forecasting/features-Dataset/demand_forecasting_features_dataset.csv", index=False)
    df.to_csv("/dbfs/FileStore/Demand-Forecasting/features-Dataset/demand_forecasting_features_dataset.csv", index=False, mode='w')

    #print("-----------")
    df_tail = df.tail(10)
    df_tail
    #print("-----------")
    # Get the column names
    #column_names = df.columns

    # Get the last 5 rows for each column
    #df_head = df.tail(5).reset_index(drop=True)

    # Align the column names with df_head
    #common_columns = column_names.intersection(df_head.columns)
    #df_head = df_head.reindex(columns=common_columns)

    # Create a MultiIndex DataFrame with common column names as the first level and column values as the second level
    #summary_df = pd.DataFrame({(col, ''): df_head[col] for col in common_columns})
   # summary_html = df_tail.to_html(index=False, border=0, classes='table table-striped')
    #restworkflowcontext.outHTML(9, title="Feature Engineered Dataset", text=summary_html)

    # Define the base download URL
    csv_file_path = f"/dbfs/FileStore/Demand-Forecasting/features-Dataset/demand_forecasting_features_dataset.csv"

    base_download_url = "https://sparkflows.kimberly-clark.com/api/v1/dbfs/files/download"
    csv_file_path_mod = f"/FileStore/Demand-Forecasting/features-Dataset/demand_forecasting_features_dataset.csv"
    file_size_bytes = 3366 #kept aribtrary small for demo purpose

    # Construct the full download URL
    download_link = f"{base_download_url}?connectionId=363&fileSize={file_size_bytes}&filePath={csv_file_path_mod}"
    restworkflowcontext.outHTML(13, title = 'Feature Engineered Dataset', text =f"<a href='{download_link}' download style='font-size: 18px; font-weight: bold; color: #FF0000;cursor: pointer;''>Feature Engineered Dataset</a>")   


    # Convert DataFrame to HTML
    #summary_html = summary_df.to_html(index=False, border=0, classes='table table-striped')


    '''
    # Get the first five rows
    df_head = df.head(5)
    # Reset the index of both DataFrames
    column_names = pd.DataFrame(column_names).reset_index(drop=True)

    df_head = df_head.reset_index(drop=True)

    # Combine column names and first five rows into a single DataFrame
    summary_df = pd.concat([column_names, df_head], axis=1)
    '''
    # Convert DataFrame to HTML
    #summary_html = summary_df.to_html(index=False, border=0, classes='table table-striped')

    #restworkflowcontext.outHTML(9, title="Sample Featured Engineer Dataset", text=summary_html)
    message = "100"
    #should_continue_execution = False 
    restworkflowcontext.outputProgress(9, title="Progress", progress=message)
    message = "Job Execution Completed without Feature Engineering."
    restworkflowcontext.outSuccess(9, title="Success", text=message)    
    #dbutils.notebook.exit("Execution stopped due to non-feature engineering case.")

In [0]:
should_continue_execution = False 
dbutils.notebook.exit("Execution stopped due to non-feature engineering case.")

In [0]:
# SMAPE: Symmetric mean absolute percentage error (adjusted MAPE)

def smape(preds, target):
    n = len(preds)
    masked_arr = ~((preds == 0) & (target == 0))
    preds, target = preds[masked_arr], target[masked_arr]
    num = np.abs(preds - target)
    denom = np.abs(preds) + np.abs(target)
    smape_val = (200 * np.sum(num / denom)) / n
    return smape_val


def lgbm_smape(preds, train_data):
    labels = train_data.get_label()
    smape_val = smape(np.expm1(preds), np.expm1(labels))
    return 'SMAPE', smape_val, False


 ## b) Time-Based Validation Sets

In [0]:
test.date.max(), test.date.min()

We selected the validation set within the range from January 1, 2017, to March 31, 2017, to best represent the time period we want to predict with our model, which covers from January 1, 2018, to March 31, 2018.

In [0]:
'''
# Train set: Data up to the beginning of 2017 (end of 2016).
train = df.loc[(df["date"] < "2017-01-01"), :]

# Validation set: The first 3 months of 2017.
val = df.loc[(df["date"] >= "2017-01-01") & (df["date"] < "2017-04-01"), :]

# Selecting features and target variables
cols = [col for col in train.columns if col not in ['date', 'id', 'sales', 'year']]

Y_train = train['sales']  # Target variable for the training set
X_train = train[cols]     # Feature variables for the training set

Y_val = val['sales']      # Target variable for the validation set
X_val = val[cols]         # Feature variables for the validation set

# Display the shapes of the target and feature variables for both training and validation sets
Y_train.shape, X_train.shape, Y_val.shape, X_val.shape
'''

## c) Time Series Model Using LightGBM

First, hyperparameters were optimized using Random Search CV, followed by Grid Search CV to find the best hyperparameters.

In [0]:
'''
# LightGBM parameters
lgb_params = {'num_leaves': 10,
              'learning_rate': 0.02,
              'feature_fraction': 0.8,
              'max_depth': 5,
              'verbose': 0,
              'num_boost_round': 1000,
              'early_stopping_rounds': 200,
              'nthread': -1}

lgbtrain = lgb.Dataset(data=X_train, label=Y_train, feature_name=cols)

lgbval = lgb.Dataset(data=X_val, label=Y_val, reference=lgbtrain, feature_name=cols)

# Train the model
model = lgb.train(
    lgb_params, 
    lgbtrain,
    valid_sets=[lgbtrain, lgbval],
    callbacks=[lgb.early_stopping(lgb_params['early_stopping_rounds'])]
)
'''

In [0]:
'''
y_pred_val = model.predict(X_val, num_iteration=model.best_iteration)

smape(np.expm1(y_pred_val), np.expm1(Y_val))
'''

## d) Feature Importance

In [0]:
'''
def plot_lgb_importances(model, plot=False, num=10):
    gain = model.feature_importance('gain')
    feat_imp = pd.DataFrame({'feature': model.feature_name(),
                             'split': model.feature_importance('split'),
                             'gain': 100 * gain / gain.sum()}).sort_values('gain', ascending=False)
    if plot:
        plt.figure(figsize=(10, 10))
        sns.set(font_scale=1)
        sns.barplot(x="gain", y="feature", data=feat_imp[0:25])
        plt.title('Feature Importance')
        plt.tight_layout()
        plt.show()
    else:
        print(feat_imp.head(num))
    return feat_imp

# Plot the top 200 feature importances and display the top 30 features
plot_lgb_importances(model, num=200)
plot_lgb_importances(model, num=30, plot=True)

# Get the feature importances
feat_imp = plot_lgb_importances(model, num=200)

# Identify features with zero importance
importance_zero = feat_imp[feat_imp["gain"] == 0]["feature"].values

# Filter out features with zero importance
imp_feats = [col for col in cols if col not in importance_zero]
len(imp_feats)
'''

When examining feature importance, it is evident that the features created during feature engineering have a significant impact on the model.

# 4) FINAL MODEL


In [0]:
'''
train = df.loc[~df.sales.isna()]
Y_train = train['sales']
X_train = train[cols]


test = df.loc[df.sales.isna()]
X_test = test[cols]

lgb_params = {'num_leaves': 10,
              'learning_rate': 0.02,
              'feature_fraction': 0.8,
              'max_depth': 5,
              'verbose': 0,
              'nthread': -1,
              "num_boost_round": model.best_iteration}

lgbtrain_all = lgb.Dataset(data=X_train, label=Y_train, feature_name=cols)

final_model = lgb.train(lgb_params, lgbtrain_all, num_boost_round=model.best_iteration)



test_preds = final_model.predict(X_test, num_iteration=model.best_iteration)
'''

### Submission File

In [0]:
'''
submission_df = test.loc[:, ["id", "sales"]]
submission_df['sales'] = np.expm1(test_preds)

submission_df['id'] = submission_df.id.astype(int)

submission_df
#submission_df.to_csv("submission_demand.csv", index=False)
'''

In [0]:
message="100"
restworkflowcontext.outputProgress(9, title="Progress", progress=message)

message = "Job Execution Completed."
restworkflowcontext.outSuccess(9, title="Success", text=message)